In [574]:
!pip install contractions
!pip install inflect

In [575]:
import pickle

def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [576]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Jameen.com")

In [577]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [578]:
import warnings
warnings.filterwarnings("ignore")

In [579]:
import pandas as pd
import numpy as np

In [580]:
# pd.set_option('display.max_colwidth', None)

In [581]:
train = pd.read_csv('../input/kaggledatafest/train.csv')
test = pd.read_csv('../input/kaggledatafest/test.csv')
sample = pd.read_csv('../input/kaggledatafest/sample_submission.csv')

In [582]:
vectorizer = TfidfVectorizer()

In [583]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

def clean_text(s):
    """
        This function cleans the text a bit
        :param s: string
        :return: cleaned string
    """
    # split by all whitespaces
    s = s.split()

    # join tokens by single space
    # why we do this?
    # this will remove all kinds of weird space
    # "hi. how are you" becomes
    # "hi. how are you"
    s = " ".join(s)

    # remove all punctuations using regex and string module
    s = re.sub(f'[{re.escape(string.punctuation)}]', '', s)

    # you can add more cleaning here if you want
    # and then return the cleaned string
    return s

def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words.split():
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

def preprocess(sample):
    sample = remove_URL(sample)
    sample = replace_contractions(sample)
    # Tokenize
#     words = nltk.word_tokenize(sample)
    
    # Normalize
    return normalize(sample)
    
def remove(text):
    return str(text).replace('[','').replace(']','').replace(',','').replace('\'','')    

In [584]:
train['Title'] = train['Title'].apply(lambda x: preprocess(x))
test['Title'] = test['Title'].apply(lambda x: preprocess(x))

train['Title'] = train['Title'].apply(lambda x: str(remove(x)))
test['Title'] = test['Title'].apply(lambda x: str(remove(x)))

In [585]:
train_tf = vectorizer.fit_transform(train['Title'])
test_tf = vectorizer.transform(test['Title'])

In [586]:
train_tf.get_shape()

(3082, 1233)

In [587]:
train = pd.concat([train, pd.DataFrame(train_tf.toarray())],axis=1)
test = pd.concat([test, pd.DataFrame(test_tf.toarray())],axis=1)

In [588]:
vecs = [1127, 1174,  464, 345, 116, 277, 685, 1025,
       1154, 407, 582, 581, 758, 1141, 1067,
       921, 593, 661, 234,  474, 480, 634,
    848, 174, 184, 638, 1187, 929, 161, 232, 1048, 221,
       1219, 542, 350, 719, 305, 1010, 106, 736, 488, 875]

In [589]:
all_cols = train.loc[:,[i for i in range(0,1233)]].sum(axis=0).sort_values(ascending=False).index
inds_to_keep = train.loc[:,[i for i in range(0,1233)]].sum(axis=0).sort_values(ascending=False)[0:139].index

train = pd.concat([train.drop(all_cols,axis=1), train[vecs]],axis=1)
test = pd.concat([test.drop(all_cols,axis=1), test[vecs]],axis=1)

In [590]:
train

,Property_Id,Title,Label,Type,Location,Baths,Area,Purpose,Bedroom(s),Description,...,1219,542,350,719,305,1010,106,736,488,875
0,381,brand new one kanal house sale bahria town gul...,"Bahria Town - Gulbahar Block, Bahria Town - Se...",House,"Bahria Town, Lahore, Punjab",6,1 Kanal,For Sale,5,Brand New 1 Kanal House For Sale In Bahria Tow...,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,1351,brand new house available sale emaar islamabad,"Emaar Canyon Views, Islamabad, Islamabad Capital",House,"Emaar Canyon Views, Islamabad, Islamabad Capital",5,1.2 Kanal,For Sale,5,A gracefully built 600 Sq. yd House is availab...,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
2,3119,one kanal house sale phase three dha defence l...,"DHA Phase 3 - Block Z, DHA Phase 3, DHA Defenc...",House,"DHA Defence, Lahore, Punjab",6,1 Kanal,For Sale,6,A nice 1 Kanal House is offered for sale. The ...,...,0.0,0.000000,0.323120,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
3,3564,luxury villa available good location,"Bahria Town - Precinct 27, Bahria Town Karachi...",House,"Bahria Town Karachi, Karachi, Sindh",3,9.4 Marla,For Sale,3,3 Bed Rooms Luxury Villa. Precinct # 27Villa S...,...,0.0,0.582959,0.000000,0.433731,0.0,0.000000,0.0,0.461241,0.000000,0.0
4,1518,five marla double storey beautiful house sale ...,"Bani Gala, Islamabad, Islamabad Capital",House,"Bani Gala, Islamabad, Islamabad Capital",3,5 Marla,For Sale,3,Newly build houseSolid constructedOwner buildH...,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3077,2036,triple storey 35x80 new house sale i8,"I-8, Islamabad, Islamabad Capital",House,"I-8, Islamabad, Islamabad Capital",10,12.4 Marla,For Sale,10,Triple Storey 35x80 New House For Sale In I-8 ...,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
3078,193,five marla house five year easy instalment pla...,"Al Jalil Garden, Lahore, Punjab",House,"Al Jalil Garden, Lahore, Punjab",3,5 Marla,For Sale,3,5 Marla Model House available in Al Jalil Gard...,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
3079,204,find one means home dha home five marla dha va...,"DHA Homes, DHA Valley, DHA Defence, Islamabad,...",House,"DHA Defence, Islamabad, Islamabad Capital",2,5 Marla,For Sale,2,Pioneers in the art of master-planned communit...,...,0.0,0.000000,0.327616,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
3080,1938,bahria enclave sector c1 ten marla full furnis...,"Bahria Enclave - Sector C1, Bahria Enclave, Ba...",House,"Bahria Town, Islamabad, Islamabad Capital",6,10 Marla,For Sale,5,Luxury interior Excellent Elevation Bahria enc...,...,0.0,0.000000,0.000000,0.000000,0.0,0.289962,0.0,0.000000,0.352532,0.0


In [591]:
import re

In [592]:
dp = load_obj('../input/dp-lat-long/dp_lat_long_fixed.pkl')
# dp = {}

In [593]:
def find_on_map(s):
    try:
        if s in dp.keys():
            return dp[s]
        location = geolocator.geocode(s)
        print(location.latitude, ' ' ,location.longitude, ' ' ,location.raw['importance'])
        dp[s] = location.latitude, location.longitude, location.raw['importance']
        return location.latitude, location.longitude, location.raw['importance']
    except:
        try:
            loc = str(re.findall(', \w+',s)[0]).replace(',','').strip() + ', Pakistan'
            location = geolocator.geocode(loc)
            print(location.latitude, ' ' ,location.longitude, ' ' ,location.raw['importance'])
            dp[s] = location.latitude, location.longitude, location.raw['importance']
            return location.latitude, location.longitude, location.raw['importance']
        except:
            dp[s] = np.nan, np.nan, np.nan
            return np.nan, np.nan, np.nan

test[['Lat','Long','Importance']] = list(test['Label'].apply(find_on_map))
train[['Lat','Long','Importance']] = list(train['Label'].apply(find_on_map))

In [594]:
def to_polar(s):
    x, y = s['Lat'], s['Long']
    rad = np.sqrt( x * x + y * y )
    theta = np.arctan(y/x)
    theta = 180 * theta/np.pi
    
    return rad, theta
    
test[['Lat','Long']] = list(test.apply(to_polar,axis=1))
train[['Lat','Long']] = list(train.apply(to_polar,axis=1))

In [595]:
train.drop(['Lat','Long','Importance'],axis=1,inplace=True)
test.drop(['Lat','Long','Importance'],axis=1,inplace=True)

In [596]:
# save_obj(dp, 'dp_lat_long_fixed')

In [597]:
train

,Property_Id,Title,Label,Type,Location,Baths,Area,Purpose,Bedroom(s),Description,...,1219,542,350,719,305,1010,106,736,488,875
0,381,brand new one kanal house sale bahria town gul...,"Bahria Town - Gulbahar Block, Bahria Town - Se...",House,"Bahria Town, Lahore, Punjab",6,1 Kanal,For Sale,5,Brand New 1 Kanal House For Sale In Bahria Tow...,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,1351,brand new house available sale emaar islamabad,"Emaar Canyon Views, Islamabad, Islamabad Capital",House,"Emaar Canyon Views, Islamabad, Islamabad Capital",5,1.2 Kanal,For Sale,5,A gracefully built 600 Sq. yd House is availab...,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
2,3119,one kanal house sale phase three dha defence l...,"DHA Phase 3 - Block Z, DHA Phase 3, DHA Defenc...",House,"DHA Defence, Lahore, Punjab",6,1 Kanal,For Sale,6,A nice 1 Kanal House is offered for sale. The ...,...,0.0,0.000000,0.323120,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
3,3564,luxury villa available good location,"Bahria Town - Precinct 27, Bahria Town Karachi...",House,"Bahria Town Karachi, Karachi, Sindh",3,9.4 Marla,For Sale,3,3 Bed Rooms Luxury Villa. Precinct # 27Villa S...,...,0.0,0.582959,0.000000,0.433731,0.0,0.000000,0.0,0.461241,0.000000,0.0
4,1518,five marla double storey beautiful house sale ...,"Bani Gala, Islamabad, Islamabad Capital",House,"Bani Gala, Islamabad, Islamabad Capital",3,5 Marla,For Sale,3,Newly build houseSolid constructedOwner buildH...,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3077,2036,triple storey 35x80 new house sale i8,"I-8, Islamabad, Islamabad Capital",House,"I-8, Islamabad, Islamabad Capital",10,12.4 Marla,For Sale,10,Triple Storey 35x80 New House For Sale In I-8 ...,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
3078,193,five marla house five year easy instalment pla...,"Al Jalil Garden, Lahore, Punjab",House,"Al Jalil Garden, Lahore, Punjab",3,5 Marla,For Sale,3,5 Marla Model House available in Al Jalil Gard...,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
3079,204,find one means home dha home five marla dha va...,"DHA Homes, DHA Valley, DHA Defence, Islamabad,...",House,"DHA Defence, Islamabad, Islamabad Capital",2,5 Marla,For Sale,2,Pioneers in the art of master-planned communit...,...,0.0,0.000000,0.327616,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
3080,1938,bahria enclave sector c1 ten marla full furnis...,"Bahria Enclave - Sector C1, Bahria Enclave, Ba...",House,"Bahria Town, Islamabad, Islamabad Capital",6,10 Marla,For Sale,5,Luxury interior Excellent Elevation Bahria enc...,...,0.0,0.000000,0.000000,0.000000,0.0,0.289962,0.0,0.000000,0.352532,0.0


In [598]:
# train['is_Banglow'] = train['Title'].apply(lambda x: 1 if 'Bungalow' in x else 0)
# test['is_Banglow'] = test['Title'].apply(lambda x: 1 if 'Bungalow' in x else 0)

In [599]:
test.head(2)

,Property_Id,Title,Label,Type,Location,Baths,Area,Purpose,Bedroom(s),Description,...,1219,542,350,719,305,1010,106,736,488,875
0,685,bungalow available sale,"Gulshan-e-Iqbal Town, Karachi, Sindh",House,"Gulshan-e-Iqbal Town, Karachi, Sindh",-,16.8 Marla,For Sale,-,Ideal Location Well Maintained 2 Sided Corner ...,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,1651,brand new double unit house sale good location,"Allama Iqbal Town - Neelam Block, Allama Iqbal...",House,"Allama Iqbal Town, Lahore, Punjab",5,5 Marla,For Sale,5,A nice and beautifully built 5 Marla House is ...,...,0.0,0.526553,0.0,0.391764,0.0,0.0,0.0,0.0,0.0,0.0


In [600]:
train[train['Price_Category']=='Affordable']

,Property_Id,Title,Label,Type,Location,Baths,Area,Purpose,Bedroom(s),Description,...,1219,542,350,719,305,1010,106,736,488,875
4,1518,five marla double storey beautiful house sale ...,"Bani Gala, Islamabad, Islamabad Capital",House,"Bani Gala, Islamabad, Islamabad Capital",3,5 Marla,For Sale,3,Newly build houseSolid constructedOwner buildH...,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
57,1625,precinct 11b villa sale bahria town karachi,"Bahria Town - Precinct 11-B, Bahria Town - Pre...",House,"Bahria Town Karachi, Karachi, Sindh",3,5 Marla,For Sale,3,Most Luxurious Houses in the heart of Bahria t...,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
73,1817,precinct 11b villa sale bahria town karachi,"Bahria Town - Precinct 11-B, Bahria Town - Pre...",House,"Bahria Town Karachi, Karachi, Sindh",3,5 Marla,For Sale,3,Most Luxurious Houses in the heart of Bahria t...,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
80,2862,five marla beautiful double storey house sale ...,"Elite Villas, Bedian Road, Lahore, Punjab",House,"Bedian Road, Lahore, Punjab",4,5 Marla,For Sale,3,Chishti Estate offers;5 Marla house for sale3 ...,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
102,2117,double story house fourteen three available sale,"I-14/3, I-14, Islamabad, Islamabad Capital",House,"I-14, Islamabad, Islamabad Capital",4,3.6 Marla,For Sale,4,A well-built House is available for sale in I-...,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,780,villa available sale,"Bahria Town - Precinct 23, Bahria Town Karachi...",House,"Bahria Town Karachi, Karachi, Sindh",-,8 Marla,For Sale,-,Villa For SaleBahria Town Villa#33 KarachiIdea...,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3033,241,house sale ghouri town phase 4a,"Ghauri Town Phase 4, Ghauri Town, Islamabad, I...",House,"Ghauri Town, Islamabad, Islamabad Capital",3,5 Marla,For Sale,3,Peaceful Location In Ghauri town Phase 4A. Sol...,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3073,258,luxury villa available good location,"Bahria Town - Precinct 27, Bahria Town Karachi...",House,"Bahria Town Karachi, Karachi, Sindh",3,9.4 Marla,For Sale,3,3 Bed Rooms Luxury Villa. Precinct # 27Villa S...,...,0.0,0.582959,0.0,0.433731,0.0,0.0,0.0,0.461241,0.0,0.0
3078,193,five marla house five year easy instalment pla...,"Al Jalil Garden, Lahore, Punjab",House,"Al Jalil Garden, Lahore, Punjab",3,5 Marla,For Sale,3,5 Marla Model House available in Al Jalil Gard...,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [601]:
# for col in test.columns:
#     train[col] = train[col].apply(lambda x: '-999_'+col if str(x).strip() == '' else x)
#     test[col] = test[col].apply(lambda x: '-999_'+col if str(x).strip() == '' else x)

In [602]:
m = {'Semi-Premium': 50000, 'Premium':100000, 'Affordable': 10000, 'Cheap': 100}

In [603]:
train['Price_Category_num'] = train['Price_Category'].map(m)

In [604]:
# train['Brand_new'] = train['Title'].apply(lambda x: 1 if (('brand' in str(x).lower()) or ('new' in str(x).lower())) else 0)
# test['Brand_new'] = test['Title'].apply(lambda x: 1 if (('brand' in str(x).lower()) or ('new' in str(x).lower())) else 0)

In [605]:
# for col in test.columns:
#     train[col] = train[col].apply(lambda x: f'empty_{col}' if str(x).strip() == '' else x)
#     test[col] = test[col].apply(lambda x: f'empty_{col}' if str(x).strip() == '' else x)

In [606]:
test.head()

,Property_Id,Title,Label,Type,Location,Baths,Area,Purpose,Bedroom(s),Description,...,1219,542,350,719,305,1010,106,736,488,875
0,685,bungalow available sale,"Gulshan-e-Iqbal Town, Karachi, Sindh",House,"Gulshan-e-Iqbal Town, Karachi, Sindh",-,16.8 Marla,For Sale,-,Ideal Location Well Maintained 2 Sided Corner ...,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,1651,brand new double unit house sale good location,"Allama Iqbal Town - Neelam Block, Allama Iqbal...",House,"Allama Iqbal Town, Lahore, Punjab",5,5 Marla,For Sale,5,A nice and beautifully built 5 Marla House is ...,...,0.0,0.526553,0.0,0.391764,0.0,0.0,0.0,0.0,0.0,0.0
2,87,house sale prime location lahore,"Sabzazar Scheme - Block A, Sabzazar Scheme, La...",House,"Sabzazar Scheme, Lahore, Punjab",6,12 Marla,For Sale,4,12 marla fully separate double storey house fo...,...,0.0,0.000000,0.0,0.488171,0.0,0.0,0.0,0.0,0.0,0.0
3,3000,sd house available sale,"Falcon Complex New Malir, Malir, Karachi, Sindh",House,"Malir, Karachi, Sindh",4,14 Marla,For Sale,4,We Are Offering You A Chance To Get Your Hands...,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,1597,ten marla brand new double unit house sale pwd...,"PWD Housing Scheme, Islamabad, Islamabad Capital",House,"PWD Housing Scheme, Islamabad, Islamabad Capital",4,10 Marla,For Sale,4,Brand New House04 Bedrooms + 04 Attached Bathr...,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [607]:
train.head()

,Property_Id,Title,Label,Type,Location,Baths,Area,Purpose,Bedroom(s),Description,...,542,350,719,305,1010,106,736,488,875,Price_Category_num
0,381,brand new one kanal house sale bahria town gul...,"Bahria Town - Gulbahar Block, Bahria Town - Se...",House,"Bahria Town, Lahore, Punjab",6,1 Kanal,For Sale,5,Brand New 1 Kanal House For Sale In Bahria Tow...,...,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,50000
1,1351,brand new house available sale emaar islamabad,"Emaar Canyon Views, Islamabad, Islamabad Capital",House,"Emaar Canyon Views, Islamabad, Islamabad Capital",5,1.2 Kanal,For Sale,5,A gracefully built 600 Sq. yd House is availab...,...,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,50000
2,3119,one kanal house sale phase three dha defence l...,"DHA Phase 3 - Block Z, DHA Phase 3, DHA Defenc...",House,"DHA Defence, Lahore, Punjab",6,1 Kanal,For Sale,6,A nice 1 Kanal House is offered for sale. The ...,...,0.000000,0.32312,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,100000
3,3564,luxury villa available good location,"Bahria Town - Precinct 27, Bahria Town Karachi...",House,"Bahria Town Karachi, Karachi, Sindh",3,9.4 Marla,For Sale,3,3 Bed Rooms Luxury Villa. Precinct # 27Villa S...,...,0.582959,0.00000,0.433731,0.0,0.0,0.0,0.461241,0.0,0.0,50000
4,1518,five marla double storey beautiful house sale ...,"Bani Gala, Islamabad, Islamabad Capital",House,"Bani Gala, Islamabad, Islamabad Capital",3,5 Marla,For Sale,3,Newly build houseSolid constructedOwner buildH...,...,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,10000


In [608]:
train['Price_Category'].value_counts()

Semi-Premium    1837
Premium         1057
Affordable       170
Cheap             18
Name: Price_Category, dtype: int64

In [609]:
train.drop(['Title'],axis=1,inplace=True)
test.drop(['Title'],axis=1,inplace=True)

In [610]:
# train[['L1', 'L2', 'L3', 'L4', 'L5']] = ''

In [611]:
# for i,vals in enumerate(train['Label'].apply(lambda x: x.split(','))):
#     for j,val in enumerate(vals):
#         train.at[i, f'L{j+1}'] = val

In [612]:
train.drop(['Label'],axis=1,inplace=True)
test.drop(['Label'],axis=1,inplace=True)

In [613]:
train[['Town', 'City', 'Province']] = ''
test[['Town', 'City', 'Province']] = ''

In [614]:
cols = ['Town', 'City', 'Province']
for i,vals in enumerate(train['Location'].apply(lambda x: x.split(','))):
    for j,val in enumerate(vals):
        v = cols[j]
        train.at[i, v] = val
        
cols = ['Town', 'City', 'Province']
for i,vals in enumerate(test['Location'].apply(lambda x: x.split(','))):
    for j,val in enumerate(vals):
        v = cols[j]
        test.at[i, v] = val

In [615]:
# train[['City_Aff','City_Cheap','City_Premium','City_Sem']] = train[['City_Aff','City_Cheap','City_Premium','City_Sem']].apply(lambda x: x/np.sum(x), axis=1)
# test[['City_Aff','City_Cheap','City_Premium','City_Sem']] = test[['City_Aff','City_Cheap','City_Premium','City_Sem']].apply(lambda x: x/np.sum(x), axis=1)

In [616]:
# m = train.groupby('Town')['Price_Category_num'].mean()

# train['Town_Avg'] = train['Town'].map(m)
# test['Town_Avg'] = test['Town'].map(m)

In [617]:
train.drop('Price_Category_num',axis=1,inplace=True)

In [618]:
train

,Property_Id,Type,Location,Baths,Area,Purpose,Bedroom(s),Description,Main Features,Rooms,...,719,305,1010,106,736,488,875,Town,City,Province
0,381,House,"Bahria Town, Lahore, Punjab",6,1 Kanal,For Sale,5,Brand New 1 Kanal House For Sale In Bahria Tow...,"Built in year: 2018 , Parking Spaces: 4 , Dou...","Bedrooms: 5 , Bathrooms: 7 , Servant Quarters...",...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,Bahria Town,Lahore,Punjab
1,1351,House,"Emaar Canyon Views, Islamabad, Islamabad Capital",5,1.2 Kanal,For Sale,5,A gracefully built 600 Sq. yd House is availab...,,"Bedrooms: 5 , Bathrooms: 5",...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,Emaar Canyon Views,Islamabad,Islamabad Capital
2,3119,House,"DHA Defence, Lahore, Punjab",6,1 Kanal,For Sale,6,A nice 1 Kanal House is offered for sale. The ...,,"Bedrooms: 6 , Bathrooms: 6 , Drawing Room , D...",...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,DHA Defence,Lahore,Punjab
3,3564,House,"Bahria Town Karachi, Karachi, Sindh",3,9.4 Marla,For Sale,3,3 Bed Rooms Luxury Villa. Precinct # 27Villa S...,,"Bedrooms: 3 , Bathrooms: 3 , Drawing Room , D...",...,0.433731,0.0,0.000000,0.0,0.461241,0.000000,0.0,Bahria Town Karachi,Karachi,Sindh
4,1518,House,"Bani Gala, Islamabad, Islamabad Capital",3,5 Marla,For Sale,3,Newly build houseSolid constructedOwner buildH...,,,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,Bani Gala,Islamabad,Islamabad Capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3077,2036,House,"I-8, Islamabad, Islamabad Capital",10,12.4 Marla,For Sale,10,Triple Storey 35x80 New House For Sale In I-8 ...,,"Bedrooms: 10 , Bathrooms: 10 , Drawing Room ,...",...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,I-8,Islamabad,Islamabad Capital
3078,193,House,"Al Jalil Garden, Lahore, Punjab",3,5 Marla,For Sale,3,5 Marla Model House available in Al Jalil Gard...,,"Bedrooms: 3 , Bathrooms: 3",...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,Al Jalil Garden,Lahore,Punjab
3079,204,House,"DHA Defence, Islamabad, Islamabad Capital",2,5 Marla,For Sale,2,Pioneers in the art of master-planned communit...,"Built in year: 2016 , Parking Spaces: 1 , Dou...","Bedrooms: 2 , Bathrooms: 2 , Kitchens: 1 , St...",...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,DHA Defence,Islamabad,Islamabad Capital
3080,1938,House,"Bahria Town, Islamabad, Islamabad Capital",6,10 Marla,For Sale,5,Luxury interior Excellent Elevation Bahria enc...,"Built in year: 2018 , Double Glazed Windows ,...","Bedrooms: 5 , Bathrooms: 6 , Servant Quarters...",...,0.000000,0.0,0.289962,0.0,0.000000,0.352532,0.0,Bahria Town,Islamabad,Islamabad Capital


In [619]:
train.drop(['Location'],axis=1,inplace=True)
test.drop(['Location'],axis=1,inplace=True)

In [620]:
for i, row in enumerate(train['Area'].iteritems()):
    if 'Marla' in str(row[1]):
        val = float(row[1].split(' ')[0].replace(',','')) / 20
        train.at[i, 'Area'] = val
        
        
for i, row in enumerate(test['Area'].iteritems()):
    if 'Marla' in str(row[1]):
        val = float(row[1].split(' ')[0].replace(',','')) / 20
        test.at[i, 'Area'] = val

In [621]:
train['Area'] = train['Area'].apply(lambda x: float(x.split(' ')[0])*5445 if type(x)==str else x*5445)
test['Area'] = test['Area'].apply(lambda x: float(x.replace(',','').split(' ')[0])*5445 if type(x)==str else x*5445)

In [622]:
# train['Est_Price'] = train['Area'] * train['City'].apply(lambda x: 12522 if x=='Karachi' else 6512 if x=='Lahore' else 4911)
# test['Est_Price'] = test['Area'] * test['City'].apply(lambda x: 12522 if x=='Karachi' else 6512 if x=='Lahore' else 4911)

In [623]:
train.drop(['Purpose'],axis=1,inplace=True)
test.drop(['Purpose'],axis=1,inplace=True)

In [624]:
train.drop(['Description'],axis=1,inplace=True)
test.drop(['Description'],axis=1,inplace=True)

In [625]:
# for i, row in enumerate(test['Main Features'].iteritems()):
#     val = str(row[1])
#     val = val.split(',')
#     for v in val:
#         try:
#             l = v.split(':')
#             col, num = l[0], l[1]
#             col = col.strip()
#             num = int(num)
#             test.at[i,col] = num
#             break
#         except:
#             pass
        
# for i, row in enumerate(train['Main Features'].iteritems()):
#     val = str(row[1])
#     val = val.split(',')
#     for v in val:
#         try:
#             l = v.split(':')
#             col, num = l[0], l[1]
#             col = col.strip()
#             num = int(num)
#             train.at[i,col] = num
#             break
#         except:
#             pass

In [626]:
# for now only
train.drop(['Main Features'],axis=1,inplace=True)
test.drop(['Main Features'],axis=1,inplace=True)

In [627]:
for i, row in enumerate(train['Rooms'].iteritems()):
    val = str(row[1])
    val = val.split(',')
    for v in val:
        try:
            l = v.split(':')
            col, num = l[0], l[1]
            col = col.strip()
            num = int(num)
            train.at[i,col] = num
        except:
            pass

In [628]:
for i, row in enumerate(test['Rooms'].iteritems()):
    val = str(row[1])
    val = val.split(',')
    for v in val:
        try:
            l = v.split(':')
            col, num = l[0], l[1]
            col = col.strip()
            num = int(num)
            test.at[i,col] = num
        except:
            pass

In [629]:
train[['Bedrooms', 'Bathrooms', 'Servant Quarters', 'Kitchens', 'Store Rooms']] = train[['Bedrooms', 'Bathrooms', 'Servant Quarters', 'Kitchens', 'Store Rooms']].fillna(0)
test[['Bedrooms', 'Bathrooms', 'Servant Quarters', 'Kitchens', 'Store Rooms']] = test[['Bedrooms', 'Bathrooms', 'Servant Quarters', 'Kitchens', 'Store Rooms']].fillna(0)

In [630]:
train.drop(['Rooms'],axis=1,inplace=True)
test.drop(['Rooms'],axis=1,inplace=True)

In [631]:
for i, row in enumerate(train['Business and Communication'].iteritems()):
    val = str(row[1])
    val = val.split(',')
    for v in val:
        try:
            col = v.strip()
            num = 1
            train.at[i,col] = num
        except:
            pass

In [632]:
for i, row in enumerate(test['Business and Communication'].iteritems()):
    val = str(row[1])
    val = val.split(',')
    for v in val:
        try:
            col = v.strip()
            if col == '': continue
            num = 1
            test.at[i,col] = num
        except:
            pass

In [633]:
train.drop(['Business and Communication'],axis=1,inplace=True)
test.drop(['Business and Communication'],axis=1,inplace=True)

In [634]:
# for i, row in enumerate(train['Healthcare Recreational'].iteritems()):
#     val = str(row[1])
#     val = val.split(',')
#     for v in val:
#         try:
#             col = v.strip()
#             if col == '': continue
#             num = 1
#             train.at[i,col] = num
#         except:
#             pass

# for i, row in enumerate(test['Healthcare Recreational'].iteritems()):
#     val = str(row[1])
#     val = val.split(',')
#     for v in val:
#         try:
#             col = v.strip()
#             if col == '': continue
#             num = 1
#             test.at[i,col] = num
#         except:
#             pass

In [635]:
#for now
train.drop(['Healthcare Recreational'],axis=1,inplace=True)
test.drop(['Healthcare Recreational'],axis=1,inplace=True)

In [636]:
for i, row in enumerate(train['Nearby Locations and Other Facilities'].iteritems()):
    val = str(row[1])
    val = val.split(',')
    for v in val:
        try:
            col = v.strip()
            num = 1
            train.at[i,col] = num
        except:
            pass
        
for i, row in enumerate(test['Nearby Locations and Other Facilities'].iteritems()):
    val = str(row[1])
    val = val.split(',')
    for v in val:
        try:
            col = v.strip()
            num = 1
            test.at[i,col] = num
        except:
            pass

In [637]:
train.drop(['Nearby Locations and Other Facilities'],axis=1,inplace=True)
test.drop(['Nearby Locations and Other Facilities'],axis=1,inplace=True)

In [638]:
# for i, row in enumerate(train['Other Facilities'].iteritems()):
#     val = str(row[1])
#     val = val.split(',')
#     for v in val:
#         try:
#             col = v.strip()
#             num = 1
#             train.at[i,col] = num
#         except:
#             pass
        
# for i, row in enumerate(test['Other Facilities'].iteritems()):
#     val = str(row[1])
#     val = val.split(',')
#     for v in val:
#         try:
#             col = v.strip()
#             num = 1
#             test.at[i,col] = num
#         except:
#             pass

In [639]:
train.drop(['Other Facilities'],axis=1,inplace=True)
test.drop(['Other Facilities'],axis=1,inplace=True)

In [640]:
train['Time Stamp'] = pd.to_datetime(train['Time Stamp'])
test['Time Stamp'] = pd.to_datetime(test['Time Stamp'])

In [641]:
train['Month'] = train['Time Stamp'].dt.month
# train['Day'] = train['Time Stamp'].dt.day

test['Month'] = test['Time Stamp'].dt.month
# test['Day'] = test['Time Stamp'].dt.day

# train['Hour'] = train['Time Stamp'].dt.hour
# test['Hour'] = test['Time Stamp'].dt.hour

In [642]:
# for now
train.drop(['Time Stamp'],axis=1,inplace=True)
test.drop(['Time Stamp'],axis=1,inplace=True)

In [643]:
train.drop(['Type'],axis=1,inplace=True)
test.drop(['Type'],axis=1,inplace=True)

In [644]:
train.drop(['Source'],axis=1,inplace=True)
test.drop(['Source'],axis=1,inplace=True)

In [645]:
train.drop(['Property_Id'],axis=1,inplace=True)
test.drop(['Property_Id'],axis=1,inplace=True)

In [646]:
train.drop(['Bedroom(s)'],axis=1,inplace=True)
test.drop(['Bedroom(s)'],axis=1,inplace=True)

In [647]:
train.drop(['Baths'],axis=1,inplace=True)
test.drop(['Baths'],axis=1,inplace=True)

In [648]:
cols_to_rem = list(test.columns)

In [649]:
cols_to_rem.append('Price_Category')

In [650]:
train = train[cols_to_rem]

In [651]:
train.isna().sum()

Area                                  0
Popular                             984
1127                                  0
1174                                  0
464                                   0
                                   ... 
Distance From Airport (kms)        1488
Nearby Public Transport Service    1389
Other Nearby Places                1691
Month                                 0
Price_Category                        0
Length: 66, dtype: int64

In [652]:
test.isna().sum()

Area                                 0
Popular                            176
1127                                 0
1174                                 0
464                                  0
                                  ... 
Nearby Restaurants                 236
Distance From Airport (kms)        264
Nearby Public Transport Service    249
Other Nearby Places                295
Month                                0
Length: 65, dtype: int64

In [653]:
train['Popular'] = train['Popular'].fillna('-999')
test['Popular'] = test['Popular'].fillna('-999')

In [654]:
train.drop(['Province'],axis=1,inplace=True)
test.drop(['Province'],axis=1,inplace=True)

In [655]:
# train = train.rename(columns={'': 'Empty'})
# test = test.rename(columns={'': 'Empty'})

In [656]:
train.drop([''],axis=1,inplace=True)
test.drop([''],axis=1,inplace=True)

In [657]:
# train['Area_Bin'] = train['Area'].apply(lambda x: 0 if x < 8 else 1)
# test['Area_Bin'] = test['Area'].apply(lambda x: 0 if x < 8 else 1)

In [658]:
# train['Area_Bin'].value_counts()

In [659]:
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

In [660]:
# train['AxA'] = train['Area'] * train['Bathrooms']
# test['AxA'] = test['Area'] * test['Bathrooms']

In [661]:
d = dict(train.groupby(['Bedrooms','Bathrooms'])['Area'].mean())

In [662]:
d

{(0.0, 0.0): 3744.606550218341,
 (0.0, 5.0): 2178.0,
 (0.0, 14.0): 13068.0,
 (2.0, 0.0): 1322.357142857143,
 (2.0, 2.0): 1532.7674999999997,
 (2.0, 3.0): 1400.142857142857,
 (2.0, 4.0): 1361.25,
 (3.0, 0.0): 1945.2701612903227,
 (3.0, 2.0): 1270.5,
 (3.0, 3.0): 2046.1856249999996,
 (3.0, 4.0): 1711.5287946428573,
 (3.0, 5.0): 1452.8250000000003,
 (3.0, 6.0): 1361.25,
 (4.0, 0.0): 3439.425,
 (4.0, 2.0): 1585.8562499999998,
 (4.0, 3.0): 1279.575,
 (4.0, 4.0): 2672.5359375000003,
 (4.0, 5.0): 2557.417500000001,
 (4.0, 6.0): 2511.182142857143,
 (4.0, 7.0): 2722.5,
 (4.0, 41.0): 3267.0,
 (5.0, 0.0): 4964.723076923077,
 (5.0, 3.0): 3539.25,
 (5.0, 4.0): 4099.307142857143,
 (5.0, 5.0): 4453.770283018868,
 (5.0, 6.0): 4561.387851782363,
 (5.0, 7.0): 3264.8506578947367,
 (5.0, 8.0): 5445.0,
 (6.0, 0.0): 5022.211764705882,
 (6.0, 5.0): 3303.2999999999997,
 (6.0, 6.0): 5026.1383333333315,
 (6.0, 7.0): 5966.726886792453,
 (6.0, 8.0): 5084.26875,
 (6.0, 9.0): 4029.3,
 (6.0, 10.0): 5445.0,
 (7.0, 0.

In [663]:
# train['Bed_Bath_Area'] = train.apply(lambda x: d[(x['Bedrooms'], x['Bathrooms'])] if ((x['Bedrooms'], x['Bathrooms']) in d.keys()) else 0, axis=1)

# test['Bed_Bath_Area'] = test.apply(lambda x: d[(x['Bedrooms'], x['Bathrooms'])] if ((x['Bedrooms'], x['Bathrooms']) in d.keys()) else 0, axis=1)

In [664]:
m = train.groupby(['City', 'Popular'])['Price_Category'].count()

In [665]:
m = dict(m)

In [666]:
m

{(' Islamabad', '-999'): 336,
 (' Islamabad', 'hot'): 627,
 (' Islamabad', 'super hot'): 103,
 (' Karachi', '-999'): 301,
 (' Karachi', 'hot'): 562,
 (' Karachi', 'super hot'): 79,
 (' Lahore', '-999'): 347,
 (' Lahore', 'hot'): 374,
 (' Lahore', 'super hot'): 353}

In [667]:
# train['City_Aff'] = train.apply(lambda x: m[(x['City'], x['Popular'])] if ((x['City'], x['Popular']) in m.keys()) else 0, axis=1)
# train['City_Cheap'] = train.apply(lambda x: m[(x['City'], x['Popular'])] if ((x['City'], x['Popular']) in m.keys()) else 0, axis=1)
# train['City_Premium'] = train.apply(lambda x: m[(x['City'], x['Popular'])] if ((x['City'], x['Popular']) in m.keys()) else 0, axis=1)
# train['City_Sem'] = train.apply(lambda x: m[(x['City'], x['Popular'])] if ((x['City'], x['Popular']) in m.keys()) else 0, axis=1)

# test['City_Aff'] = test.apply(lambda x: m[(x['City'], x['Popular'])] if ((x['City'], x['Popular']) in m.keys()) else 0, axis=1)
# test['City_Cheap'] = test.apply(lambda x: m[(x['City'], x['Popular'])] if ((x['City'], x['Popular']) in m.keys()) else 0, axis=1)
# test['City_Premium'] = test.apply(lambda x: m[(x['City'], x['Popular'])] if ((x['City'], x['Popular']) in m.keys()) else 0, axis=1)
# test['City_Sem'] = test.apply(lambda x: m[(x['City'], x['Popular'])] if ((x['City'], x['Popular']) in m.keys()) else 0, axis=1)

In [668]:
# train['x'] = train['Area'] / 4
# train['y'] = train['Area'] - (train['Area'] / 4)

# test['x'] = test['Area'] / 4
# test['y'] = test['Area'] - (test['Area'] / 4)

In [669]:
train.drop(['Other Business and Communication Facilities'],axis=1,inplace=True)
test.drop(['Other Business and Communication Facilities'],axis=1,inplace=True)

In [670]:
train

,Area,Popular,1127,1174,464,345,116,277,685,1025,...,Intercom,Nearby Schools,Nearby Hospitals,Nearby Shopping Malls,Nearby Restaurants,Distance From Airport (kms),Nearby Public Transport Service,Other Nearby Places,Month,Price_Category
0,5445.00,super hot,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,3,Semi-Premium
1,6534.00,hot,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,Semi-Premium
2,5445.00,super hot,0.000000,0.0,0.0,0.0,0.0,0.0,0.349328,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,Premium
3,2559.15,-999,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,Semi-Premium
4,1361.25,hot,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,Affordable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3077,3375.90,-999,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,Premium
3078,1361.25,super hot,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,Affordable
3079,1361.25,-999,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3,Cheap
3080,2722.50,super hot,0.244076,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,3,Semi-Premium


In [671]:
train.to_csv('train_clean.csv', index = False)
test.to_csv('test_clean.csv', index = False)

In [672]:
from category_encoders import TargetEncoder, MEstimateEncoder, JamesSteinEncoder, LeaveOneOutEncoder
from sklearn.preprocessing import LabelEncoder

In [673]:
le = LabelEncoder()
train['Price_Category'] = le.fit_transform(train['Price_Category'])

In [674]:
te = JamesSteinEncoder()

In [675]:
cols = train.columns[train.dtypes=='object']

In [676]:
cols

Index(['Popular', 'Town', 'City'], dtype='object')

In [677]:
train[cols] = te.fit_transform(train[cols], train['Price_Category'])

In [678]:
test[cols] = te.transform(test[cols])

In [679]:
train

,Area,Popular,1127,1174,464,345,116,277,685,1025,...,Intercom,Nearby Schools,Nearby Hospitals,Nearby Shopping Malls,Nearby Restaurants,Distance From Airport (kms),Nearby Public Transport Service,Other Nearby Places,Month,Price_Category
0,5445.00,2.476636,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,3,3
1,6534.00,2.451056,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
2,5445.00,2.476636,0.000000,0.0,0.0,0.0,0.0,0.0,0.349328,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,2
3,2559.15,2.527439,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,3
4,1361.25,2.451056,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3077,3375.90,2.527439,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,2
3078,1361.25,2.476636,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0
3079,1361.25,2.527439,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3,1
3080,2722.50,2.476636,0.244076,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,3,3


In [680]:
X = train.drop(['Price_Category'], axis=1)
y = train['Price_Category'].copy()

In [681]:
# from sklearn.cluster import KMeans

# from yellowbrick.cluster import KElbowVisualizer


# # Instantiate the clustering model and visualizer
# model = KMeans()
# visualizer = KElbowVisualizer(model, k=(2,12))

# visualizer.fit(X[['Lat','Long']], y)        # Fit the data to the visualizer
# visualizer.show()   

In [682]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE

In [683]:
def Generate_Data(X, y):
    oversample = SMOTE()
    X, y = oversample.fit_resample(X, y)
    return X, y

In [684]:
def cross_val(X, y, model, metric = f1_score, folds = 10):
    scores = []
    for tr_in, val_in in StratifiedKFold(n_splits = folds).split(X, y):
        model_fold = model
        
        X_train, y_train, X_val, y_val = X.iloc[tr_in,:], y[tr_in], X.iloc[val_in,:], y[val_in]
#         X_train, y_train = Generate_Data(X_train, y_train)
        
        model_fold.fit(X_train, y_train)
        y_hat = model.predict(X_val)
        scores.append(metric(y_val,y_hat, average='macro'))
    return np.mean(scores)

In [685]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [686]:
# X.drop(ld,axis=1,inplace=True)
# test.drop(ld,axis=1,inplace=True)

In [687]:
y.value_counts()

3    1837
2    1057
0     170
1      18
Name: Price_Category, dtype: int64

In [688]:
from collections import Counter

In [689]:
counter = Counter(y)
print(counter)

Counter({3: 1837, 2: 1057, 0: 170, 1: 18})


In [690]:
sum_ = np.sum(list(counter.values()))

In [691]:
weights = {0: 1 - (counter[0] / sum_), 1: 1 - (counter[1] / sum_), 2: 1 - (counter[2] / sum_),
          3: 1 - (counter[3] / sum_)}

In [692]:
weights

{0: 0.9448410123296561,
 1: 0.9941596365996106,
 2: 0.6570408825438028,
 3: 0.4039584685269305}

In [693]:
lgbm = LGBMClassifier(n_jobs=-1)
rf = RandomForestClassifier(n_jobs=-1,class_weight=weights)
xgb = XGBClassifier(n_estimators = 100, n_jobs=-1, verbosity = 0,sample_weight=weights)
cat = CatBoostClassifier(verbose=0, class_weights=weights)
extra = ExtraTreesClassifier(n_jobs=-1, class_weight=weights)

In [694]:
# best_sc = 0.77

# for c1 in X.columns:
#     lgbm = LGBMClassifier(n_jobs=-1)
#     cat = ExtraTreesClassifier(n_jobs=-1)
#     xgb = XGBClassifier(n_jobs=-1, verbosity = 0)

#     X_ = X.copy()
#     X_.drop(c1, axis=1, inplace=True)
#     score = cross_val_ensemble(X_, y, [lgbm, cat, xgb], folds=10)
#     if score > best_sc:
#         print(f'{c1} = {score}')
#     #                 best_sc = score

In [695]:
# best_sc = 0.77

# for c1 in X.columns:
#     for c2 in X.columns:
#         lgbm = LGBMClassifier(n_jobs=-1)
#         cat = ExtraTreesClassifier(n_jobs=-1)
#         xgb = XGBClassifier(n_jobs=-1, verbosity = 0)

#         X_ = X.copy()
#         X_['nf'] = (X_[c1] + X_[c2]) 
#         score = cross_val_ensemble(X_, y, [lgbm, cat, xgb], folds=10)
#         if score > best_sc:
#             print(f'{c1} + {c2} = {score}')
#         #                 best_sc = score

In [696]:
train

,Area,Popular,1127,1174,464,345,116,277,685,1025,...,Intercom,Nearby Schools,Nearby Hospitals,Nearby Shopping Malls,Nearby Restaurants,Distance From Airport (kms),Nearby Public Transport Service,Other Nearby Places,Month,Price_Category
0,5445.00,2.476636,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,3,3
1,6534.00,2.451056,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
2,5445.00,2.476636,0.000000,0.0,0.0,0.0,0.0,0.0,0.349328,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,2
3,2559.15,2.527439,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,3
4,1361.25,2.451056,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3077,3375.90,2.527439,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,2
3078,1361.25,2.476636,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0
3079,1361.25,2.527439,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3,1
3080,2722.50,2.476636,0.244076,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,3,3


In [697]:
cross_val(X, y, lgbm)

0.7728596586205008

In [698]:
cross_val(X, y, rf)

0.7139095149448809

In [699]:
cross_val(X, y, xgb)

0.7633910869738779

In [700]:
# cross_val(X, y, cat)

In [701]:
cross_val(X, y, extra)

0.6893790182910424

In [702]:
def mode(s):
    lst = [s[1], s[2], s[3]]
    return max(set(lst), key=lst.count)

In [703]:
def cross_val_ensemble(X, y, models, metric = f1_score, folds = 10):
    scores = []
    for tr_in, val_in in StratifiedKFold(n_splits = folds).split(X, y):
        model_1, model_2, model_3 = models[0], models[1], models[2]
        X_train, y_train, X_val, y_val = X.iloc[tr_in,:], y[tr_in], X.iloc[val_in,:], y[val_in]
        
        model_1.fit(X_train, y_train)
        model_2.fit(X_train, y_train)
        model_3.fit(X_train, y_train)
        
        preds = pd.concat([pd.DataFrame(model_1.predict(X_val)), pd.DataFrame(model_2.predict(X_val)), 
                           pd.DataFrame(model_3.predict(X_val))],axis=1)
        preds.columns = [1,2,3]
        
        y_hat = preds.apply(mode, axis=1)
        
        scores.append(metric(y_val,y_hat, average='macro'))
    return np.mean(scores)

In [704]:
lgbm = LGBMClassifier(n_jobs=-1)
rf = RandomForestClassifier(n_jobs=-1,class_weight=weights)
xgb = XGBClassifier(n_estimators = 100, n_jobs=-1, verbosity = 0,sample_weight=weights)
cat = CatBoostClassifier(verbose=0, class_weights=weights)
extra = ExtraTreesClassifier(n_jobs=-1, class_weight=weights)

In [717]:
cross_val_ensemble(X, y, [lgbm, rf, xgb])

0.7758358276748579

In [706]:
lgbm = LGBMClassifier(n_jobs=-1)
rf = RandomForestClassifier(n_jobs=-1,class_weight=weights)
xgb = XGBClassifier(n_estimators = 100, n_jobs=-1, verbosity = 0,sample_weight=weights)
cat = CatBoostClassifier(verbose=0, class_weights=weights)
extra = ExtraTreesClassifier(n_jobs=-1, class_weight=weights)

In [707]:
# model.fit(X, y)
lgbm.fit(X, y)
rf.fit(X, y)
xgb.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1,
              sample_weight={0: 0.9448410123296561, 1: 0.9941596365996106,
                             2: 0.6570408825438028, 3: 0.4039584685269305},
              scale_pos_weight=None, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=0)

In [708]:
imps = pd.DataFrame()
imps['cols'] = X.columns
imps['importance1'] = xgb.feature_importances_
imps['importance2'] = rf.feature_importances_
imps['importance3'] = lgbm.feature_importances_

In [709]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
imps['importance'] = np.nan
imps['importance'] = ss.fit_transform(imps[['importance1','importance2','importance3']])

In [710]:
imps = imps.sort_values('importance', ascending = False).reset_index(drop = True)

In [711]:
imps['cols'].head(50).values

array(['Area', 1174, 'Town', 1127, 581, 277, 'City', 1067, 685, 758,
       'Month', 464, 407, 116, 634, 1187, 929, 345, 474, 1025, 234, 542,
       'Bedrooms', 1154, 305, 'Bathrooms', 736, 'Store Rooms', 582, 848,
       'Kitchens', 1141, 875, 1219, 'Servant Quarters', 638, 161, 480,
       593, 1048, 174, 350, 221, 661, 184, 488, 232, 'Nearby Hospitals',
       921, 'Nearby Public Transport Service'], dtype=object)

In [712]:
preds = pd.concat([pd.DataFrame(xgb.predict(test)), pd.DataFrame(rf.predict(test)), 
                           pd.DataFrame(lgbm.predict(test))],axis=1)
preds.columns = [1,2,3]

y_hat = preds.apply(mode, axis=1)

sample['Price_Category'] = le.inverse_transform(y_hat)

In [713]:
sample['Price_Category'].value_counts()

Semi-Premium    318
Premium         205
Affordable       22
Name: Price_Category, dtype: int64

In [714]:
sample

,Property_Id,Price_Category
0,685,Premium
1,1651,Semi-Premium
2,87,Semi-Premium
3,3000,Premium
4,1597,Semi-Premium
...,...,...
540,300,Semi-Premium
541,1934,Semi-Premium
542,2306,Semi-Premium
543,2097,Semi-Premium


In [715]:
sample.to_csv('submission.csv', index = False)

In [716]:
test

,Area,Popular,1127,1174,464,345,116,277,685,1025,...,Satellite or Cable TV Ready,Intercom,Nearby Schools,Nearby Hospitals,Nearby Shopping Malls,Nearby Restaurants,Distance From Airport (kms),Nearby Public Transport Service,Other Nearby Places,Month
0,4573.80,2.451056,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,1361.25,2.476636,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,3267.00,2.476636,0.000000,0.0,0.0,0.0,0.0,0.0,0.479472,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,3811.50,2.451056,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5
4,2722.50,2.527439,0.290328,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,2722.50,2.451056,0.353279,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3
541,1361.25,2.476636,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
542,1633.50,2.527439,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5
543,2722.50,2.451056,0.244286,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3
